In [2]:
from __future__ import print_function
import torch as t
from torch.autograd import Variable as V

In [6]:
# grad_fn可以查看这个variable的反向传播函数
# z是add函数的输出，所以它的反向传播函数是AddBackward
x = V(t.ones(1))
b = V(t.rand(1), requires_grad = True)
w = V(t.rand(1), requires_grad = True)
y = w * x # 等价于y=w.mul(x)
z = y + b # 等价于z=y.add(b)
z.creator # 书上的版本用 z.grad_fn 在此我用creator代替grad_fn

In [11]:
# 后面的一部分巴拉巴拉的，我的PyTorch版本没有grad_fn进行不了，省略省略

In [13]:
def abs(x):
    if x.data[0]>0: return x
    else: return -x
x = V(t.ones(1),requires_grad=True)
y = abs(x)
y.backward()
x.grad

Variable containing:
 1
[torch.FloatTensor of size 1]

In [14]:
x = V(-1*t.ones(1),requires_grad=True)
y = abs(x)
y.backward()
print(x.grad)

Variable containing:
-1
[torch.FloatTensor of size 1]



In [16]:
def f(x):
    result = 1
    for ii in x:
        if ii.data[0]>0: result=ii*result
    return result
x = V(t.arange(-2,4),requires_grad=True)
y = f(x) # y = x[3]*x[4]*x[5]
y.backward()
x.grad

Variable containing:
 0
 0
 0
 6
 3
 2
[torch.FloatTensor of size 6]

In [17]:
# volatile=True是另外一个很重要的标志，它能够将所有依赖于它的节点全部设置为volatile=True，其优先级比requires_grad=True高。
# volatile=True的节点不会求导，即使requires_grad=True，也不会进行反向传播。
# 对于不需要反向传播的情景（如inference，测试推理时），该参数可实现一定程度的速度提升，并节省显存。
x = V(t.ones(1))
w = V(t.rand(1), requires_grad=True)
y = x * w
# y依赖于w，而w.requires_grad = True
x.requires_grad, w.requires_grad, y.requires_grad

(False, True, True)

In [18]:
x = V(t.ones(1), volatile=True)
w = V(t.rand(1), requires_grad = True)
y = x * w
# y依赖于w和x，但x.volatile = True, w.requires_grad = True
x.requires_grad, w.requires_grad, y.requires_grad

(False, True, False)